In [1]:
!pip install -q transformers
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast


2025-04-22 16:08:29.697237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745338109.933070      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745338110.010144      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [3]:
from datasets import load_dataset

# Load SQuAD dataset
dataset = load_dataset("squad", split="train")

# Step 1: Add a new column for context length
dataset = dataset.map(lambda x: {"context_length": len(x["context"].split())})
dataset = dataset.map(lambda x: {"question_length": len(x["question"].split())})


# Step 2: Sort by the new column
sorted_dataset = dataset.sort("context_length")

# Step 3: Select the shortest 10k rows (or whatever range you need)
subset = sorted_dataset.select(range(11000))  # You can change this to 5000, 20000, etc.
df = subset.to_pandas()


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
example = df.iloc[0]
question = example["question"]
context = example["context"]
answer_text = example["answers"]["text"][0]
answer_start = example["answers"]["answer_start"][0]
answer_end = answer_start + len(answer_text)

# Tokenize with offset mapping to map char→token index
encoding = tokenizer(
    question,
    context,
    return_offsets_mapping=True,
    padding="max_length",
    truncation=True,
    max_length=384,
    return_tensors="tf"
)

# Find token start and end positions from char offset
offsets = encoding["offset_mapping"][0].numpy()
start_token = end_token = 0

for idx, (start, end) in enumerate(offsets):
    if start <= answer_start < end:
        start_token = idx
    if start < answer_end <= end:
        end_token = idx
        break

print(f"Question: {question}")
print(f"Answer: {answer_text}")
print(f"Predicted Start Token Index: {start_token}")
print(f"Predicted End Token Index: {end_token}")


Question: How many groups of modern hunter-gatherers are there?
Answer: Only a few
Predicted Start Token Index: 1
Predicted End Token Index: 3


I0000 00:00:1745338226.732130      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745338226.732975      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [6]:
def prepare_qa_data(df, tokenizer, max_len=384):
    input_ids = []
    attention_masks = []
    start_positions = []
    end_positions = []

    for i, row in df.iterrows():
        question = row["question"]
        context = row["context"]
        answer_text = row["answers"]["text"][0]
        answer_start = row["answers"]["answer_start"][0]
        answer_end = answer_start + len(answer_text)

        encoding = tokenizer(
            question,
            context,
            return_offsets_mapping=True,
            padding="max_length",
            truncation=True,
            max_length=max_len,
            return_tensors=None  # gives plain dict
        )

        offsets = encoding["offset_mapping"]
        start_pos, end_pos = 0, 0

        for idx, (start, end) in enumerate(offsets):
            if start <= answer_start < end:
                start_pos = idx
            if start < answer_end <= end:
                end_pos = idx
                break

        input_ids.append(encoding["input_ids"])
        attention_masks.append(encoding["attention_mask"])
        start_positions.append(start_pos)
        end_positions.append(end_pos)

    return {
        "input_ids": np.array(input_ids),
        "attention_mask": np.array(attention_masks),
        "start_positions": np.array(start_positions),
        "end_positions": np.array(end_positions),
    }


In [7]:
train_data = prepare_qa_data(df, tokenizer)

print("input_ids shape:", train_data["input_ids"].shape)
print("start_positions shape:", train_data["start_positions"].shape)


input_ids shape: (11000, 384)
start_positions shape: (11000,)


In [14]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_data["input_ids"],
        "attention_mask": train_data["attention_mask"]
    },
    {
        "start_positions": train_data["start_positions"],
        "end_positions": train_data["end_positions"]
    }
))

train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [15]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim):
        super().__init__()
        self.token_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embedding = tf.keras.layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length

    def call(self, inputs):
        positions = tf.range(start=0, limit=self.sequence_length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.position_embedding(positions)
        return embedded_tokens + embedded_positions


In [16]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.1):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            tf.keras.layers.Dense(dense_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization()
        self.layernorm2 = tf.keras.layers.LayerNormalization()
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, mask=None, training=False):
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        out1 = self.layernorm1(inputs + self.dropout(attention_output, training=training))
        dense_output = self.dense_proj(out1)
        return self.layernorm2(out1 + self.dropout(dense_output, training=training))


In [17]:
def build_qa_transformer_model(
    vocab_size,
    sequence_length,
    embed_dim=128,
    dense_dim=512,
    num_heads=4,
    num_layers=2
):
    input_ids = tf.keras.Input(shape=(sequence_length,), dtype="int32", name="input_ids")
    attention_mask = tf.keras.Input(shape=(sequence_length,), dtype="int32", name="attention_mask")

    # Step 1: Positional embedding
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(input_ids)

    # ✅ Step 2: Fix this line — expand attention mask dims using Lambda
    mask = tf.keras.layers.Lambda(
        lambda x: tf.cast(x[:, tf.newaxis, tf.newaxis, :], dtype=tf.float32)
    )(attention_mask)

    # Step 3: Transformer encoder stack
    for _ in range(num_layers):
        x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x, mask)

    start_logits = tf.keras.layers.Dense(1)(x)
    start_logits = tf.keras.layers.Reshape((sequence_length,), name="start_positions")(start_logits)

    end_logits = tf.keras.layers.Dense(1)(x)
    end_logits = tf.keras.layers.Reshape((sequence_length,), name="end_positions")(end_logits)


    # Final model
    model = tf.keras.Model(
        inputs=[input_ids, attention_mask],
        outputs={"start_positions": start_logits, "end_positions": end_logits}
    )
    return model


In [22]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model = build_qa_transformer_model(
    vocab_size=len(tokenizer.vocab),
    sequence_length=384,
    embed_dim=256,
    dense_dim=256,
    num_heads=4,
    num_layers=2  # You can make this 3 max
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss={"start_positions": loss_fn, "end_positions": loss_fn},
    metrics={"start_positions": "accuracy", "end_positions": "accuracy"}
)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 384)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 384)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_2    │ (None, 384, 256)       │      7,911,936 │ input_ids[0][0]        │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 1, 1, 384)      │              0 │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_4     │ (None, 384, 256)       │      1,184,512 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_5     │ (None, 384, 256)       │      1,184,512 │ transformer_encoder_4… │
│ (TransformerEncoder)      │                        │                │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 384, 1)         │            257 │ transformer_encoder_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 384, 1)         │            257 │ transformer_encoder_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ end_positions (Reshape)   │ (None, 384)            │              0 │ dense_17[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ start_positions (Reshape) │ (None, 384)            │              0 │ dense_16[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,281,474 (39.22 MB)

 Trainable params: 10,281,474 (39.22 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
print(model.output_names)


ListWrapper(['end_positions', 'start_positions'])


In [24]:
history = model.fit(
    train_dataset,
    epochs=15,  # You can increase this later
    verbose=1
)


Epoch 1/15


I0000 00:00:1745339017.763956     117 service.cc:148] XLA service 0x37d77060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745339017.766482     117 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745339017.766505     117 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745339018.740851     117 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/344 ━━━━━━━━━━━━━━━━━━━━ 1:45:45 19s/step - end_positions_accuracy: 0.0000e+00 - end_positions_loss: 6.8419 - loss: 12.0975 - start_positions_accuracy: 0.0000e+00 - start_positions_loss: 5.2556

I0000 00:00:1745339026.253892     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


344/344 ━━━━━━━━━━━━━━━━━━━━ 67s 141ms/step - end_positions_accuracy: 0.0050 - end_positions_loss: 5.8227 - loss: 11.6038 - start_positions_accuracy: 0.0056 - start_positions_loss: 5.7811
Epoch 2/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 40s 116ms/step - end_positions_accuracy: 0.0034 - end_positions_loss: 5.9579 - loss: 11.9173 - start_positions_accuracy: 0.0027 - start_positions_loss: 5.9594
Epoch 3/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 40s 116ms/step - end_positions_accuracy: 0.0022 - end_positions_loss: 5.9539 - loss: 11.9074 - start_positions_accuracy: 0.0030 - start_positions_loss: 5.9535
Epoch 4/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 40s 117ms/step - end_positions_accuracy: 0.0023 - end_positions_loss: 5.9530 - loss: 11.9043 - start_positions_accuracy: 0.0028 - start_positions_loss: 5.9513
Epoch 5/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 40s 116ms/step - end_positions_accuracy: 0.0021 - end_positions_loss: 5.9522 - loss: 11.9051 - start_positions_accuracy: 0.0038 - start_positions_loss: 5.9529
Epoch 6/15
344/3

In [25]:
def predict_answer(question, context, tokenizer, model, max_len=384):
    # Tokenize the input like BERT: [CLS] question [SEP] context [SEP]
    inputs = tokenizer(
        question,
        context,
        return_tensors='tf',
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_offsets_mapping=True
    )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    offset_mapping = inputs['offset_mapping'][0].numpy()
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Run the model
    outputs = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})

    # Get logits
    # start_logits = outputs[0][0]  
    start_logits = outputs['start_positions'][0]
    # end_logits = outputs[1][0]    
    end_logits= outputs['end_positions'][0]

    # Get predicted start and end indices
    start_idx = np.argmax(start_logits)
    end_idx = np.argmax(end_logits)

    # Handle edge case: if end before start, fallback to start only
    if end_idx < start_idx:
        end_idx = start_idx

    # Convert token indices to character positions
    start_char = offset_mapping[start_idx][0]
    end_char = offset_mapping[end_idx][1]

    # Return the actual text span from the context
    return context[start_char:end_char]

In [26]:
question = "Which year did the USSR cancel the N1 rocket program?"
context = "Meanwhile, the USSR continued briefly trying to perfect their N1 rocket, finally canceling it in 1976..."

predicted_answer = predict_answer(question, context, tokenizer, model)
print("Predicted Answer:", predicted_answer)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Answer: 


In [27]:
test_subset = subset.select(range(10000, 11000))  # rows 10000 to 10999 (inclusive)

In [28]:
test_df = test_subset.to_pandas()
test_data = prepare_qa_data(test_df, tokenizer)

In [29]:
def evaluate_model_on_test_set(model, test_df, tokenizer):
    correct = 0
    total = len(test_df)

    for i, row in test_df.iterrows():
        question = row["question"]
        context = row["context"]
        true_answer = row["answers"]["text"][0]

        predicted_answer = predict_answer(question, context, tokenizer, model)

        # Simple match (you can use fuzzy matching or partial overlap later)
        if predicted_answer.strip().lower() == true_answer.strip().lower():
            correct += 1

    accuracy = correct / total
    return accuracy


In [30]:
accuracy = evaluate_model_on_test_set(model, test_df, tokenizer)
print(f"Evaluation Accuracy on Test Set (Exact Match): {accuracy:.2%}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━